Segmentación del mercado de adolecentes en sparklyr
===

* *30 min* | Última modificación: Junio 22, 2019

En este tutorial se aplica el algoritmo K-means para clasificar un grupo de adolecentes con base en sus intéreses, con el fin de diseñar estrategias publicitarias y servicios encaminados a cada grupo de interés usando SparkR. Este tutorial se enfoca en la programación de Sparklyr y no en el análisis del problema. Para abordar este tutorial, el lector debe tener suficiencia en los módulos correspondientes de analítica predictiva.

## Definición del problema

Un vendedor desea enviar publicidad electrónica a una población de adolecentes y adultos jóvenes con el fin de maximizar sus ventas. Para ello, desea poder clasificar a sus clientes potenciales por grupos de interés de acuerdo con sus intereses y consecuentemente enviar publicidad específica a cada uno de ellos.   

En este problema se desea determina que grupos de interés existen en una población de clientes a partir de los mensajes enviados por un servicio de redes sociales. La información disponible consiste en 30000 observaciones de 40 variables que podrían caracterizar los intereses de la población analizada. Estas variables corresponden a palabras que pueden asociarse a un interés de la poblaión analizada. Cada variable mide la frecuencia con que una determinada palabra aparece en los mensajes de texto; adicionalmente, dentro de estas variables se incluye  información como el sexo, la edad y la cantidad de contactos de la persona. 

## Solución

### Preparación

In [1]:
##
## Esta función se usará para ejecutar comandos en el 
## sistema operativo y capturar la salida.
##
systemp <- function(command) cat(system(command, intern = TRUE), sep = '\n')

In [2]:
library(sparklyr)
library(dplyr)
spark_installed_versions()
sc <- spark_connect(master='local', spark_home='/home/vagrant/spark/spark-2.4.3-bin-hadoop2.7')
spark_version(sc)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



spark,hadoop,dir
<chr>,<chr>,<chr>
2.4.3,2.7,/home/vagrant/spark/spark-2.4.3-bin-hadoop2.7


[1] ‘2.4.3’

### Carga de datos

El archivo con los datos se encuentra en la carpeta actual de trabajo en la máquina local.

In [3]:
## copia el archivo al HDFS
systemp('hdfs dfs -copyFromLocal snsdata.csv /tmp/snsdata.csv') 

Warning message:
“running command 'hdfs dfs -copyFromLocal insurance.csv /tmp/snsdata.csv' had status 1”

In [4]:
df <- 
spark_read_csv(sc,                  ## spark_connection
               'snsdata',           ## nombre de la tabla
               '/tmp/snsdata.csv')  ## ubicación del archivo
                                    ## en el sistema hdfs
df

# Source: spark<snsdata> [?? x 40]
   gradyear gender   age friends basketball football soccer softball volleyball
      <int> <chr>  <dbl>   <int>      <int>    <int>  <int>    <int>      <int>
 1     2006 M       19.0       7          0        0      0        0          0
 2     2006 F       18.8       0          0        1      0        0          0
 3     2006 M       18.3      69          0        1      0        0          0
 4     2006 F       18.9       0          0        0      0        0          0
 5     2006 NA      19.0      10          0        0      0        0          0
 6     2006 F      NaN       142          0        0      0        0          0
 7     2006 F       18.9      72          0        0      0        0          0
 8     2006 M       18.3      17          0        0      0        1          0
 9     2006 F       19.1      52          0        0      0        0          0
10     2006 F       18.7      39          0        0      0        0          0
# … w

In [5]:
##
## Cantidad de registros leidos
##
count(df)

# Source: spark<?> [?? x 1]
      n
  <dbl>
1 30000

### Análisis exploratorio

A continuación se ejemplifican algunos cómputos típicos para el análisis exploratorio.

In [6]:
##
## Conteo por género
##
df %>%  group_by(gender) %>% summarize(count = n())

# Source: spark<?> [?? x 2]
  gender count
  <chr>  <dbl>
1 M       5222
2 F      22054
3 NA      2724

In [7]:
##
## Conteo por género
##
table(df %>% select(gender) %>% collect)


    F     M    NA 
22054  5222  2724 

In [8]:
##
## Estadísticos por rango de edades
##
summary(df %>% select(age) %>% collect())

      age         
 Min.   :  3.086  
 1st Qu.: 16.312  
 Median : 17.287  
 Mean   : 17.994  
 3rd Qu.: 18.259  
 Max.   :106.927  
 NA's   :5086     

In [9]:
##
## Estadísticos por rango de edades
##
sdf_describe(df, 'age')

# Source: spark<?> [?? x 2]
  summary age               
  <chr>   <chr>             
1 count   24914             
2 mean    17.993949546439772
3 stddev  7.858054477853863 
4 min     3.086             
5 max     106.927           

Los métodos que pueden aplicarse a las columnas están listados en: https://spark.rstudio.com/reference/

In [10]:
##
## Cantidad de nulos en la columna age
##
df %>% filter(isNull(age)) %>% count

# Source: spark<?> [?? x 1]
      n
  <dbl>
1  5086

In [11]:
##
## Se agrega una columna con las edades entre 13 y 19,
## reemplazando por null los valores por fuera de este
## rango
##
df <- 
mutate(df, 
       age1319 = ifelse(age >= 13 & age < 20, age, NA))

##
## Se verifican los valores en la nueva columna
##
sdf_describe(df, 'age1319')

# Source: spark<?> [?? x 2]
  summary age1319           
  <chr>   <chr>             
1 count   24477             
2 mean    17.25242893328433 
3 stddev  1.1574649278955391
4 min     13.027            
5 max     19.995            

In [12]:
##
## Se calcula la edad promedio por año 
## de graduación para la muestra en el
## rango de edades considerado
##
age_df <- df %>% group_by(gradyear) %>% summarize(avg_age = avg(age))

age_df %>% collect

gradyear,avg_age
<int>,<dbl>
2009,16.87602
2006,19.13724
2007,18.39146
2008,17.52387


### Entrenamiento del modelo

In [13]:
##
## Agrupamiento usando k-means
##
model <- 
ml_kmeans(
    df,
    ~ friends + basketball + football + soccer + softball + 
      volleyball + swimming + cheerleading + baseball + tennis + 
      sports + cute + sex + sexy + hot + kissed + dance + band + 
      marching + music + rock + god + church + jesus + bible + 
      hair + dress + blonde + mall + shopping + clothes + hollister + 
      abercrombie + die + death + drunk + drugs,
    k = 5,
    max_iter = 100,
    seed = 12345,
    prediction_col = 'cluster'
)

In [14]:
##
## Pronóstico del cluster de cada grupo
##
fitted <- ml_predict(model, df) %>% collect

### Análisis del modelo

In [15]:
##
## Cantidad de patrones por cluster
##
fitted %>%  group_by(cluster) %>% summarize(count = n())

cluster,count
<int>,<int>
0,3502
1,16886
2,7
3,8994
4,611
